# Factor Analysis on Pythia training logs

Eleuther ran some evals during training that are available in their pythia repo.
Here we load and factorise that data.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.decomposition import PCA, FactorAnalysis
from lsoc.factor import factor, selection, vis, data
import numpy as np

In [ ]:
from lsoc.factor import data

models = [
    "pythia-70m",
    "pythia-160m",
    "pythia-410m",
    "pythia-1.4b",
    "pythia-2.8b",
    "pythia-6.9b",
    "pythia-12b",
]
output_path = data.default_path + "/evals/eleuther"

model_dfs = {}
for model in models:
    print(model)
    path = f"{output_path}/{model}.csv"
    model_dfs[model] = pd.read_csv(path, index_col=0)

In [ ]:
# Extract the final checkpoint (it's going to be at most 7-dimensional, but I bet it's lower)
final = 143000
extract = {}
for model in models:
    extract[model] = model_dfs[model].loc[final]

    #row_means = model_dfs[model].mean(axis=1)
    # Find the index of the row with the highest average value
    # max_mean_idx = row_means.idxmax()
    # Extract the row with the highest average value
    #extract[model] = model_dfs[model].loc[max_mean_idx]

mini_psych = pd.DataFrame(extract).T
print("Psychometrics Matrix (EOT)")
mini_psych


In [ ]:
# Its strongly one-dimensional - I guess that makes sense because that factor is model-size

#scaled = StandardScaler().fit_transform(mini_psych)
scaled = mini_psych
model = factor.PCA(iters=50)  # inhouse leaveout recipe
# model = factor.FA(iters=20)  # look, they generally agree!
errs = selection.cross_validate(scaled, model, max_factors=3, n_folds=10, repeats=1)
fig = vis.crossval(*errs, method_name=model.name)
# Improve layout
fig.update_layout(
    xaxis_title='Number of factors',
    yaxis_title='Reconstruction Error',
    showlegend=True
)
fig.show()

In [ ]:
# Try what do the factor analysis factors look like?

factr = FactorAnalysis(1)
factr.fit(mini_psych)

# Create a figure
fig = go.Figure()

# Add a trace for each model
for model in models:
    Xp = model_dfs[model]
    yp = -factr.transform(Xp)[:,0]
    st = Xp.index
    # Add trace with model name annotation
    fig.add_trace(
        go.Scatter(
            x=st,
            y=yp,
            mode='lines+markers',
            name=model,
            hovertemplate= model + '<br>Step: %{x}<br>Effective Size: %{y:.1e}'
        )
    )

    # Update layout with proper labels and log scales
fig.update_layout(
    title='Factor Analysis (1 factor final checkpoint)',
    xaxis_title='Step',
    yaxis_title='Latent Factor',
    xaxis_type='log',  # Set x-axis to log scale
    # yaxis_type='log',  # Set y-axis to log scale
    legend_title='Models',
    hovermode='closest',
    template='plotly_white',
    width=900,
    height=600,
    margin=dict(l=80, r=80, t=100, b=80)
)

fig.show()

In [ ]:
# Which components have the highest loading?
components = np.abs(factr.components_[0])
keep = components > 1e-8

w = pd.DataFrame(
    [components[keep]],
    columns=mini_psych.columns.values[keep],
)
w.T.sort_values(by=0, ascending=False).T

Copyright (c) Gradient Institute and Timaeus. All rights reserved.

Licensed under the Apache 2.0 License.
